# Basketize: Optimizing Basketball Rosters

Basketize is an optimization model whose main objective is to minimize an NBA team's salary while achieving key team statistics likely to lead to wins. As a follow-up to my [HiddenBasket](https://www.linkedin.com/pulse/hiddenbasket-most-undervalued-nba-players-xavier-lim-2f/) model which took an individualistic approach to roster creation, Basketize is a more holistic (team) approach to creating an undervalued team. Rather than finding the 12 most undervalued players and putting them on a team, Basketize determines which combination of 12 players will collectively lead the NBA in all team statistics likely to lead to wins.

Prior to developing the model, a correlation analysis will be performed to determine which team statistics are highly correlated with winning during the 2019-2020 season. Then, among the 30 NBA teams, the best team at each of those team statistics will be determined. The value for each of those team statistics will be used as model constraints to ensure the players selected by the model will collectively lead the NBA in all of those areas. Finally, the Basketize model will be used to determine which free agent players the Toronto Raptors should target this coming 2020 offseason to maximize their chance of winning.

#    
# Packages

In [490]:
using ExcelFiles
using DataFrames
using JuMP
using Cbc
using Statistics
using StatsBase

#  
# Data Sets

For my project, I will collect three data sets from [Basketball Reference](https://www.basketball-reference.com/):
* The **Player Statistics** data set presents information and performance statistics about each player such as their position (Pos), team (Tm), games played (G), minutes played (MP), field goal percentage (FGp), 3-pointers per game (P3), 3-point percentage (P3p), effective field goal percentage (eFGp), free throws per game (FT), free throw attempts per game (FTA), free throw percentage (FTp), offensive rebounds per game (ORB), defensive rebounds per game (DRB), total rebounds per game (TRB), assists per game (AST), turnovers per game (TOV), points per game (PTS), true shooting percentage (TSp), 3-point attempt rate (P3Ar), defensive win shares (DWS), and defensive box plus/minus (DBPM).

  - Effective field goal percentage (eFGp) is the percentage of field goals a player makes and adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.
  - True shooting percentage (TSp) measures shooting efficiency that takes into account field goals, 3-point field goals, and free throws.
  - 3-point attempt rate (P3Ar) is the percentage of field goal attempts from 3-point range.
  - Defensive win shares (DWS) estimates the number of wins contributed by a player due to his defense.
  - Defensive box plus/minus (DBPM) estimates a player’s defensive contribution to the team when that player is on the court.


* The **Player Salaries** data set presents the salaries of all NBA players for the 2019-2020 to 2023-2024 season.


* The **Team Statistics** data set presents performance statistics about each team such as their win-loss percentage (WLp), field goal percentage (FGp), 3-pointers per game (P3), 3-point percentage (P3p), free throws per game (FT), free-throw attempts per game (FTA), free throw percentage (FTp), offensive rebounds per game (ORB), defensive rebounds per game (DRB), total rebounds per game (TRB), assists per game (AST), turnovers per game (TOV), points per game (PTS), 3-point attempt rate (P3Ar), pace, true shooting percentage (TSp), effective field goal percentage (eFGp), offensive rating (ORtg), and defensive rating (DRtg). 
     - Win-loss percentage (WLp) is the percentage of games a team wins. I decided to use this statistic rather than simply using wins to quantify winning because some teams played more games than others prior to the COVID-19 play stoppage.
     - Offensive rating (ORtg) shows how many points a team scores per 100 possessions.
     - Defensive rating (DRtg) shows how many points a team allows per 100 possessions. 

# Data Wrangling / Cleaning

There are numerous data wrangling procedures I must complete prior to developing the model. First, I will select only the columns relevant to my analysis from all data sets. Then, I will replace all missing values with 0 because I am assuming a field will only be empty if a player did not qualify for that statistic. For example, a player who has never shot a 3-pointer will have a missing value for their 3-point percentage. Next, I will remove players who have played less than 10 games in the 2019-2020 season from the Player Statistics data set due to playing too few games. I am choosing to set the cutoff at 10 games because if a player played less than 10 games, they would have played less than 15% of the season which is too small of a sample. After removing these players from the data set, I will join the Player Statistics and Player Salaries data sets to show each player's performance statistics accompanied by their annual salary from 2019-2020 to 2023-2024. This final compiled data set will be called **Player Statistics & Salaries**.

It is also important to note that prior to loading the data sets into Julia, I changed all column names that started with a number or had a percentage sign because Julia does not allow calling columns with either of these attributes.

I also want to mention that since there is no commonly used accurate measurement of defense, it is quite difficult to use statistics to evaluate a player's defensive ability. Thus, I am choosing not to use basic defensive metrics such as steals and blocks because both metrics have deficiencies when trying to analyze a player's defensive ability. For example, many players who average a lot of steals may not necessarily be good defenders since they tend to play the passing lanes and gamble which could leave their defensive assignment wide open. Although defensive win shares and defensive box-plus minus have a few deficiencies, I believe they do a better job of measuring defense than simply using steals and blocks.

### Player Statistics

In [491]:
# Load Player Statistics Data Set
playerStats = DataFrame(load("nba_data_2019_2020.xlsx", "Player Statistics"))

# Select Columns Relevant to Analysis
playerStats = playerStats[:,["Player","Pos","Tm","G","MP","FGp","P3","P3p","eFGp","FT","FTA","FTp","ORB","DRB","TRB","AST",
                            "TOV","PTS","TSp","P3Ar","DWS","DBPM"]]

# Replace Missing Values with 0
playerStats = coalesce.(playerStats, 0)

# Select Players Who Played More Than 10 Games
validPlayerStats = playerStats[playerStats.G .>= 10,:]

first(validPlayerStats,10)

,Player,Pos,Tm,G,MP,FGp,P3,P3p
,String,String,String,Float64,Float64,Float64,Float64,Float64
1,Steven Adams,C,OKC,58.0,27.0,0.591,0.0,0.333
2,Bam Adebayo,PF,MIA,65.0,34.4,0.567,0.0,0.077
3,LaMarcus Aldridge,C,SAS,53.0,33.1,0.493,1.2,0.389
4,Nickeil Alexander-Walker,SG,NOP,41.0,12.2,0.339,1.0,0.342
5,Grayson Allen,SG,MEM,30.0,16.6,0.449,1.1,0.363
6,Jarrett Allen,C,BRK,64.0,25.7,0.646,0.0,0.0
7,Kadeem Allen,SG,NYK,10.0,11.7,0.432,0.5,0.313
8,Al-Farouq Aminu,PF,ORL,18.0,21.1,0.291,0.5,0.25
9,Kyle Anderson,PF,MEM,59.0,19.3,0.493,0.3,0.258


### Player Salaries 

In [492]:
# Load Player Salaries Data Set
playerSalaries = DataFrame(load("nba_data_2019_2020.xlsx", "Salaries"))

# Select Columns Relevant to Analysis
playerSalaries = playerSalaries[:,["Player", "Salary2019_20", "Salary2020_21", "Salary2021_22", "Salary2022_23", 
                                    "Salary2023_24"]]

first(playerSalaries,10)

,Player,Salary2019_20,Salary2020_21,Salary2021_22,Salary2022_23,Salary2023_24
,String,Float64,Float64?,Float64?,Float64?,Float64?
1,Stephen Curry,4.02318e7,4.30064e7,4.5781e7,missing,missing
2,Chris Paul,3.85065e7,4.13588e7,4.42111e7,missing,missing
3,Russell Westbrook,3.8178e7,4.1006e7,4.3848e7,4.6662e7,missing
4,John Wall,3.78e7,4.0824e7,4.3848e7,4.6872e7,missing
5,James Harden,3.78e7,4.0824e7,4.3848e7,4.6872e7,missing
6,LeBron James,3.74369e7,3.92196e7,4.10023e7,missing,missing
7,Kevin Durant,3.7199e7,3.9059e7,4.09189e7,4.27788e7,missing
8,Blake Griffin,3.4235e7,3.6596e7,3.8957e7,missing,missing
9,Kyle Lowry,3.32963e7,3.0e7,missing,missing,missing


Players without a signed contract for a specific season is labelled as "*missing*". Thus, each player will become a free agent the first season they have a missing salary. For example, Stephen Curry will become a free agent after the 2021-2022 season.

It is important to note that certain players have a player or team option in certain years. For more information, please visit: [Basketball Reference - NBA Player Contracts](https://www.basketball-reference.com/contracts/players.html) 

### Player Statistics & Salaries

In [493]:
# Join the Player Statistics and Player Salaries Data Sets
playerData = sort(innerjoin(validPlayerStats, playerSalaries, on = :Player, makeunique=true))

first(playerData,10)

,Player,Pos,Tm,G,MP,FGp,P3,P3p
,String,String,String,Float64,Float64,Float64,Float64,Float64
1,Aaron Gordon,PF,ORL,58.0,33.0,0.433,1.2,0.301
2,Aaron Holiday,PG,IND,58.0,23.6,0.407,1.4,0.394
3,Abdel Nader,SF,OKC,48.0,15.8,0.455,0.8,0.371
4,Admiral Schofield,SF,WAS,27.0,10.9,0.4,0.5,0.298
5,Al Horford,C,PHI,60.0,30.8,0.442,1.5,0.337
6,Al-Farouq Aminu,PF,ORL,18.0,21.1,0.291,0.5,0.25
7,Alec Burks,SG,TOT,59.0,27.3,0.404,1.7,0.366
8,Alen Smailagić,PF,GSW,14.0,9.9,0.5,0.2,0.231
9,Alex Caruso,PG,LAL,58.0,17.8,0.423,0.7,0.355



### Team Statistics

In [494]:
# Load Team Statistics Data Set
teamStats = DataFrame(load("nba_data_2019_2020.xlsx", "Team Statistics"))

# Select Columns Relevant to Analysis
teamStats = teamStats[:,["Team","WLp","FGp","P3","P3p","FT","FTA","FTp","ORB","DRB","TRB","AST","TOV","PTS","P3Ar","Pace",
                        "TSp","eFGp","ORtg", "DRtg"]]

first(teamStats,10)

,Team,WLp,FGp,P3,P3p,FT,FTA,FTp
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Atlanta Hawks,0.299,0.449,12.0,0.333,18.5,23.4,0.79
2,Boston Celtics,0.672,0.459,12.4,0.363,18.3,22.8,0.801
3,Brooklyn Nets,0.469,0.444,12.9,0.34,18.0,24.1,0.744
4,Charlotte Hornets,0.354,0.434,12.1,0.352,16.2,21.6,0.748
5,Chicago Bulls,0.338,0.447,12.2,0.348,15.5,20.5,0.755
6,Cleveland Cavaliers,0.292,0.458,11.2,0.351,15.1,19.9,0.758
7,Dallas Mavericks,0.597,0.462,15.3,0.369,17.9,23.1,0.773
8,Denver Nuggets,0.662,0.471,10.9,0.358,15.9,20.5,0.775
9,Detroit Pistons,0.303,0.459,12.0,0.367,16.6,22.4,0.743


#   
# Correlation Between Team Statistics and Winning 

Before developing the model, I want to see which team statistics are highly correlated with win/loss percentage (WLp) to determine which team statistics are generally correlated with winning. Then, I will incorporate these team statistics into the constraints of my optimization model.

### Shooting Statistics 

In [495]:
Statistics.cor(Array(teamStats[:,[:WLp, :FGp, :eFGp, :TSp, :FT, :FTA , :FTp, :P3, :P3p, :P3Ar]]))

10×10 Array{Float64,2}:
 1.0         0.689017    0.753582  …   0.282948     0.451154    0.158433
 0.689017    1.0         0.841701     -0.04627      0.61058    -0.247523
 0.753582    0.841701    1.0           0.490377     0.727469    0.296467
 0.731793    0.78827     0.954643      0.484772     0.71866     0.304677
 0.199622    0.00892738  0.178444      0.325141     0.0469758   0.331365
 0.183686   -0.037505    0.15977   …   0.371762    -0.0449432   0.400147
 0.0960784   0.116533    0.107017      0.00296387   0.220396   -0.0493335
 0.282948   -0.04627     0.490377      1.0          0.324536    0.943549
 0.451154    0.61058     0.727469      0.324536     1.0         0.0519972
 0.158433   -0.247523    0.296467      0.943549     0.0519972   1.0

As you can see from the first column, winning (WLp) is:

* Strongly correlated with Field Goal Percentage (FGp), Effective Field Goal Percentage (eFGp), and True Shooting Percentage (TSp)
* Moderately correlated with 3-Pointers Per Game (P3) and 3-Point Percentage (P3p)
* Weakly correlated with 3-Point Attempt Rate (P3Ar), Free Throws Per Game (FT), Free Throw Attempts Per Game (FTA), and Free Throw Percentage (FTp)

Since Effective Field Goal Percentage (**eFGp**) has the highest correlation with winning and is highly correlated with the other shooting percentages, I will include it in my model to represent shooting percentages in general. Despite 3-Pointers Per Game (**P3**) and 3-Point Percentage (**P3p**) only being moderately correlated with winning, I will include them in my model because of the growing importance of 3-point shooting in the NBA over the last couple years.

### General Offensive Statistics

In [496]:
Statistics.cor(Array(teamStats[:,[:WLp, :PTS, :AST, :TOV, :Pace]]))

5×5 Array{Float64,2}:
  1.0         0.548474  0.151006  -0.281933  0.0688078
  0.548474    1.0       0.147627  -0.034426  0.73421
  0.151006    0.147627  1.0        0.181112  0.193911
 -0.281933   -0.034426  0.181112   1.0       0.36226
  0.0688078   0.73421   0.193911   0.36226   1.0

As you can see from the first column, winning (WLp) is:

* Strongly correlated with Points Per Game (PTS)
* Moderately correlated with Turnovers (TOV)
* Weakly correlated with Assists (AST) and Pace

I will include Points Per Game (**PTS**) in my model since it is strongly correlated with winning.

### Rebounding Statistics 

In [497]:
Statistics.cor(Array(teamStats[:,[:WLp, :ORB, :DRB, :TRB]]))

4×4 Array{Float64,2}:
  1.0       -0.242412    0.648027   0.528056
 -0.242412   1.0        -0.0987819  0.293603
  0.648027  -0.0987819   1.0        0.922102
  0.528056   0.293603    0.922102   1.0

As you can see from the first column, winning (WLp) is:

* Strongly correlated with Defensive Rebounding (DRB) and Total Rebounding (TRB)
* Moderately correlated with Offensive Rebounding (ORB)

Since Defensive Rebounding (**DRB**) has the highest correlation with winning and is highly correlated with the Total Rebounding (TRB), I will include it in my model to represent rebounding in general.

### Offensive and Defensive Rating 

In [498]:
Statistics.cor(Array(teamStats[:,[:WLp, :ORtg, :DRtg]]))

3×3 Array{Float64,2}:
  1.0        0.762857  -0.859602
  0.762857   1.0       -0.42703
 -0.859602  -0.42703    1.0

As you can see from the first column, winning (WLp) is:

* Strongly correlated with Defensive Rating (DRtg) and Offensive Rating (ORtg)

Interestingly, defensive rating has a stronger correlation with winning than offensive rating. Thus, it is vital to include defensive metrics in my model.

Although Basketball Reference has data on team Defensive Rating (DRtg), it unfortunately does not have data on individual player Defensive Rating. Thus, as a substitute for Defensive Rating, I will incorporate Defensive Win Shares (**DWS**) and Defensive-Box Plus Minus (**DBPM**) in my model.

#    

Taken together, winning teams tend to shoot efficiently, get a lot of rebounds (especially defensive rebounds), score a lot of points (especially three pointers), and hold opposing teams to very few points, which all intuitively make sense.

Thus, I will incorporate the following statistics into the constraints of my model: Effective Field Goal Percentage (eFGp), 3-Pointers Per Game (P3), 3-Point Percentage (P3p), Points Per Game (PTS), Defensive Rebounds Per Game (DRB), Defensive Win Shares (DWS), and Defensive Box-Plus Minus (DBPM).

#   
# Determining Constraint Values 

Now that I have determined which variables to include in my model constraints, I must determine the cutoff value for each constraint. I have decided to set the maximum value of each team statistic as the cutoff for each corresponding constraint. In other words, I want my team to shoot more 3 pointers, get more defensive rebounds, and score more points than any team in the NBA. I also want them to shoot more efficiently than any other team.

In [499]:
# Store Maximum Value of All Team Variables
P3_max = maximum(teamStats[!,:P3])
DRB_max = maximum(teamStats[!,:DRB])
PTS_max = maximum(teamStats[!,:PTS])
P3p_max = maximum(teamStats[!,:P3p])
eFGp_max = maximum(teamStats[!,:eFGp])

# Display Maximum Values of All Team Variables
describe(teamStats[:,[:P3,:DRB,:PTS,:P3p,:eFGp]], :max)

,variable,max
,Symbol,Float64
1,P3,15.4
2,DRB,42.2
3,PTS,118.6
4,P3p,0.383
5,eFGp,0.553


Since I will be using Defensive Win Shares (DWS) and Defensive-Box Plus Minus (DBPM) which are individual statistics, rather than taking the maximum of each statistic, I will set the 75th percentile (3rd quartile) of DWS and DBPM as their respective cutoff constraint values. This will ensure that the average player selected by my model will be an elite defender.

In [500]:
# Store 75th percentile of DWS and DBPM
DWS_q75 = percentile(playerData[!,:DWS], 75)
DBPM_q75 = percentile(playerData[!,:DBPM], 75)

# Display 75th percentile of DWS and DBPM
describe(playerData[:, [:DWS,:DBPM]], :q75)

,variable,q75
,Symbol,Float64
1,DWS,1.6
2,DBPM,0.9


Now that I have the values for my constraints, I will create the model.

#    
# Basketize Model

Basketize utilizes mixed-integer optimization (MIO) which is a mathematical technique that optimizes an objective function (primary goal) while satisfying certain constraints where certain variables take integer values and other variables take continuous values.  MIO consists of three components: 
* An objective function which is either maximized or minimized
* Variables which control the objective function and decide the output
* Constraints that allows the variables to take certain values while excluding others (limitations of the decision variables)

First, I will define the decision integer (binary) variable as:

$$
    x_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is selected to the team,}\\
                  0 & \text{if player $i$ is not selected to the team}
                \end{array}
              \right.
$$

Next, I will set the objective function to minimize the collective salary of the players selected to be on the team. I'll denote player $i$'s salary as $o_i$. Thus, the objective function is defined as:

$$ min \sum_{i=1}^{N}(x_i*o_i)$$

subject to the following constraints:

$ \sum_{i=1}^{N}(x_i) = 12$

$ \sum_{i=1}^{N}(x_i*t_i) \geq 15.4$ where $t_i$ is player $i$'s 3 Pointers Per Game 

$ \sum_{i=1}^{N}(x_i*u_i) \geq 42.2$ where $u_i$ is player $i$'s Defensive Rebounds Per Game

$ \sum_{i=1}^{N}(x_i*p_i) \geq 118.6$ where $p_i$ is player $i$'s Points Per Game

     

$ \frac{1}{12} \sum_{i=1}^{N}(x_i*h_i) \geq 0.383$ where $h_i$ is player $i$'s 3-Point Percentage

$ \frac{1}{12} \sum_{i=1}^{N}(x_i*v_i) \geq 0.553$ where $v_i$ is player $i$'s Effective Field Goal Percentage

       

$ \frac{1}{12} \sum_{i=1}^{N}(x_i*b_i) \geq 0.9$ where $b_i$ is player $i$'s Defensive Box Plus Minus

$ \frac{1}{12} \sum_{i=1}^{N}(x_i*w_i) \geq 1.6$ where $w_i$ is player $i$'s Defensive Win Shares

These constraints will ensure my optimized dream team consisting of 12 players will lead the NBA in three-pointers, defensive rebounding, and points per game. In addition, they will lead the league in team three-point and effective field goal percentage. Finally, the team average for defensive win shares and defensive box-plus minus will be among the highest in the NBA.

I also want to include minutes per game and position constraints in my model. Specifically, I want to have at least two players at each position. I also want the players at each position to collectively play approximately 48 minutes per game. For example, the minutes and position constraints for centers would be:

$ \sum_{i=1}^{N}(x_i*c_i) \geq 2$

$ \sum_{i=1}^{N}(x_i*m_i*c_i) \leq 48$

where $m_i$ is player $i$'s minutes per game and 
$$
    c_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is a center,}\\
                  0 & \text{if player $i$ is not a center}
                \end{array}
              \right.
$$

In [501]:
function basketize_team(df)
    model = Model(Cbc.Optimizer)
    set_silent(model)
    
    # Define N to Represent the Index of Each Player
    N = nrow(df)
    @variable(model, x[1:N], Bin)
    
    # Objective Function
    o = df[!,:Salary2019_20];
    @objective(model, Min, sum(x[i]*o[i] for i=1:N))
    
    # Team Size Constraint
    @constraint(model, sum(x[i] for i=1:N) == 12);
    
    # Position Constraints
    c = ifelse.(df[!,:Pos].=="C",1,0);
    @constraint(model, sum(x[i]*c[i] for i=1:N) >= 2);

    d = ifelse.(df[!,:Pos].=="PF",1,0);
    @constraint(model, sum(x[i]*d[i] for i=1:N) >= 2);
    
    e = ifelse.(df[!,:Pos].=="SF",1,0);
    @constraint(model, sum(x[i]*e[i] for i=1:N) >= 2);
    
    f = ifelse.(df[!,:Pos].=="SG",1,0);
    @constraint(model, sum(x[i]*f[i] for i=1:N) >= 2);
    
    g = ifelse.(df[!,:Pos].=="PG",1,0);
    @constraint(model, sum(x[i]*g[i] for i=1:N) >= 2);
    
    # Minutes Constraint
    m = df[!,:MP]
    @constraint(model, sum(x[i]*m[i]*c[i] for i=1:N) <= 48);
    @constraint(model, sum(x[i]*m[i]*d[i] for i=1:N) <= 48);
    @constraint(model, sum(x[i]*m[i]*e[i] for i=1:N) <= 48);
    @constraint(model, sum(x[i]*m[i]*f[i] for i=1:N) <= 48);
    @constraint(model, sum(x[i]*m[i]*g[i] for i=1:N) <= 48);
    
    
    # 3-Pointers Constraint
    t = df[!,:P3]
    @constraint(model, sum(x[i]*t[i] for i=1:N) >= P3_max);
    
    # Defensive-Rebounds Constraint
    u = df[!,:DRB]
    @constraint(model, sum(x[i]*u[i] for i=1:N) >= DRB_max);
    
    # Points Constraint
    p = df[!,:PTS]
    @constraint(model, sum(x[i]*p[i] for i=1:N) >= PTS_max);
    
    # 3-Point Percentage Constraint
    h = df[!,:P3p]
    @constraint(model, 1/12*sum(x[i]*h[i] for i=1:N) >= P3p_max);
    
    # Effective Field Goal Percentage Constraint
    v = df[!,:eFGp]
    @constraint(model, 1/12*sum(x[i]*v[i] for i=1:N) >= eFGp_max);
    
    # Defensive Box Plus-Minus Constraint
    b = df[!,:DBPM]
    @constraint(model, 1/12*sum(x[i]*b[i] for i=1:N) >= DBPM_q75);
    
    # Defensive Win Shares Constraint
    w = df[!,:DWS]
    @constraint(model, 1/12*sum(x[i]*w[i] for i=1:N) >= DWS_q75);

    optimize!(model)
    
    # Extract Selected Players
    players = sort(df[value.(x).>0.99,:Player])
    return(df[value.(x).>0.99,[:Player,:Pos,:MP,:eFGp,:P3p,:P3,:PTS,:DRB,:DBPM,:DWS,:Salary2019_20]])
end

basketize_team (generic function with 2 methods)

### Optimized Dream Team 

In [502]:
# Extract Model Selected Players With Their Statistics
basketized_roster = basketize_team(playerData)

,Player,Pos,MP,eFGp,P3p,P3,PTS,DRB
,String,String,Float64,Float64,Float64,Float64,Float64,Float64
1,Chris Boucher,PF,13.2,0.512,0.283,0.5,6.3,2.7
2,DeAndre Jordan,C,22.0,0.666,0.0,0.0,8.3,7.5
3,Donte DiVincenzo,SG,23.1,0.546,0.344,1.3,9.4,3.9
4,Duncan Robinson,SF,30.0,0.665,0.448,3.7,13.3,3.1
5,Jayson Tatum,PF,34.6,0.523,0.398,2.8,23.6,6.1
6,Justin Holiday,SG,24.9,0.581,0.424,1.9,8.4,2.7
7,Kyle O'Quinn,C,9.9,0.535,0.304,0.3,3.3,2.5
8,Luka Dončić,PG,33.3,0.531,0.318,2.9,28.7,8.0
9,Mfiondu Kabengele,C,5.3,0.578,0.45,0.8,3.5,0.8


In [503]:
# Compute Optimized Dream Team Payroll
sum(basketized_roster.Salary2019_20)

4.539646e7

Despite leading the NBA in almost all team statistics related to winning, the payroll of my optimized dream team would only be about \\$45 Million which is about \\$80 million less than the average NBA team.

#  
# Optimizing Toronto Raptors Roster via Free Agency 

Now, I will apply the model to a real-life problem all NBA teams face every year which is determining which players to sign in free agency. I will apply the model to the Toronto Raptors, but it could be applied to any of the 30 NBA teams.

### NBA Free Agents 

Before determining which free agents the Raptors should try to acquire, I must first determine which players will become free agents in the 2020 offseason. These players will be those without a contract for the 2020-2021 season.

In [504]:
# Find Free Agents
freeAgents = playerData[ismissing.(playerData[!,:Salary2020_21]),:]
first(freeAgents,10)

,Player,Pos,Tm,G,MP,FGp,P3,P3p,eFGp
,String,String,String,Float64,Float64,Float64,Float64,Float64,Float64
1,Alec Burks,SG,TOT,59.0,27.3,0.404,1.7,0.366,0.475
2,Alex Len,C,TOT,49.0,18.3,0.556,0.3,0.25,0.579
3,Alize Johnson,SF,IND,13.0,4.2,0.467,0.0,0.0,0.467
4,Allen Crabbe,SG,TOT,37.0,17.6,0.356,1.0,0.303,0.458
5,Allonzo Trier,PG,NYK,24.0,12.1,0.481,0.8,0.358,0.571
6,Amile Jefferson,PF,ORL,18.0,4.1,0.357,0.0,0.0,0.357
7,Ante Žižić,C,CLE,22.0,10.0,0.569,0.0,0.0,0.569
8,Anthony Tolliver,PF,TOT,47.0,15.6,0.348,0.8,0.322,0.474
9,Anžejs Pasečņiks,C,WAS,24.0,16.9,0.528,0.0,0.0,0.528


Now that I have a data frame with all the free agents, I need a data frame of all the players the Raptors have already signed for the 2020-2021 season.

### Non-Free Agent Raptors 

In [505]:
# Select All Players on the Toronto Raptors
raptors = playerData[playerData[!,:Tm] .== "TOR", :]

# Select Non-Free Agent Raptors
signedRaptors = raptors[.!ismissing.(raptors[!,:Salary2020_21]),:]

# Store Number of Non-Free Agent Raptors
numOfRaptors = length(signedRaptors[!,:Player])

signedRaptors

,Player,Pos,Tm,G,MP,FGp,P3,P3p,eFGp
,String,String,String,Float64,Float64,Float64,Float64,Float64,Float64
1,Kyle Lowry,PG,TOR,52.0,36.6,0.417,2.9,0.354,0.52
2,Matt Thomas,SG,TOR,33.0,9.7,0.486,1.1,0.467,0.644
3,Norman Powell,SG,TOR,44.0,28.8,0.502,2.1,0.398,0.593
4,OG Anunoby,SF,TOR,63.0,30.1,0.507,1.3,0.381,0.584
5,Pascal Siakam,PF,TOR,53.0,35.5,0.459,2.2,0.359,0.517
6,Patrick McCaw,SF,TOR,37.0,24.5,0.414,0.6,0.324,0.485
7,Stanley Johnson,PF,TOR,20.0,4.8,0.258,0.1,0.154,0.29
8,Terence Davis,SG,TOR,64.0,17.0,0.463,1.4,0.396,0.575


### Eligible Player Pool (All Free Agents and Non-Free Agent Raptors) 

Now I will combine the free agent players data frame with all the signed Raptors to create an eligible pool of players the Raptors can pick from this offseason.

In [506]:
# Combine Signed Raptors and Free Agent Data Sets
playerPool = append!(signedRaptors, freeAgents)
first(playerPool,10)

,Player,Pos,Tm,G,MP,FGp,P3,P3p,eFGp
,String,String,String,Float64,Float64,Float64,Float64,Float64,Float64
1,Kyle Lowry,PG,TOR,52.0,36.6,0.417,2.9,0.354,0.52
2,Matt Thomas,SG,TOR,33.0,9.7,0.486,1.1,0.467,0.644
3,Norman Powell,SG,TOR,44.0,28.8,0.502,2.1,0.398,0.593
4,OG Anunoby,SF,TOR,63.0,30.1,0.507,1.3,0.381,0.584
5,Pascal Siakam,PF,TOR,53.0,35.5,0.459,2.2,0.359,0.517
6,Patrick McCaw,SF,TOR,37.0,24.5,0.414,0.6,0.324,0.485
7,Stanley Johnson,PF,TOR,20.0,4.8,0.258,0.1,0.154,0.29
8,Terence Davis,SG,TOR,64.0,17.0,0.463,1.4,0.396,0.575
9,Alec Burks,SG,TOT,59.0,27.3,0.404,1.7,0.366,0.475


Now, I will apply the same model I used to get my optimized dream team, but force it to select the 8 players the Raptors have already signed for the 2020-2021 season. Thus, the model will pick out the remaining 4 players from the pool of free agents that will optimize the team's performance given the constraints.

$$
    r_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ plays for the Raptors,}\\
                  0 & \text{if player $i$ does not play for the Raptors}
                \end{array}
              \right.
$$

       
   
$$
    s_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is signed for the 2020-2021 season,}\\
                  0 & \text{if player $i$ will become a free agent prior to the 2020-2021 season}
                \end{array}
              \right.
$$

Ideally, I would like to use the same constraints, however, it would result in no solution because the constraints are too strict. Thus, I will remove the minutes per game constraints in order to get a solution.

### Raptors Basketize Model 

In [507]:
function basketize_Raptors(df)
    model = Model(Cbc.Optimizer)
    set_silent(model)
    
    # Define N to Represent the Index of Each Player
    N = nrow(df)
    @variable(model, x[1:N], Bin)
    
    # Objective Function
    o = df[!,:Salary2019_20];
    @objective(model, Min, sum(x[i]*o[i] for i=1:N))
    
    # Team Size Constraint
    @constraint(model, sum(x[i] for i=1:N) == 12);
    
    # Select Signed Raptors
    r = ifelse.((df[!,:Tm].== "TOR"),1,0);
    s = ifelse.((.!ismissing.(df[!,:Salary2020_21])),1,0);
    @constraint(model, sum(x[i]*r[i]*s[i] for i=1:N) == numOfRaptors);
    
    # Position Constraints
    c = ifelse.(df[!,:Pos].=="C",1,0);
    @constraint(model, sum(x[i]*c[i] for i=1:N) >= 2);
    
    d = ifelse.(df[!,:Pos].=="PF",1,0);
    @constraint(model, sum(x[i]*d[i] for i=1:N) >= 2);
    
    e = ifelse.(df[!,:Pos].=="SF",1,0);
    @constraint(model, sum(x[i]*e[i] for i=1:N) >= 2);
    
    f = ifelse.(df[!,:Pos].=="SG",1,0);
    @constraint(model, sum(x[i]*f[i] for i=1:N) >= 2);
    
    g = ifelse.(df[!,:Pos].=="PG",1,0);
    @constraint(model, sum(x[i]*g[i] for i=1:N) >= 2);
    
    # 3-Pointers Constraint
    t = df[!,:P3]
    @constraint(model, sum(x[i]*t[i] for i=1:N) >= P3_max);
    
    # Defensive Rebounds Constraint
    u = df[!,:DRB]
    @constraint(model, sum(x[i]*u[i] for i=1:N) >= DRB_max);
    
    # Points Constraint
    p = df[!,:PTS]
    @constraint(model, sum(x[i]*p[i] for i=1:N) >= PTS_max);
    
    # 3-Point Percentage Constraint
    h = df[!,:P3p]
    @constraint(model, 1/12*sum(x[i]*h[i] for i=1:N) >= P3p_max);
    
    # Effective Field Goal Percentage Constraint
    l = df[!,:eFGp]
    @constraint(model, 1/12*sum(x[i]*l[i] for i=1:N) >= eFGp_max);
    
    # Defensive Box Plus-Minus Constraint
    b = df[!,:DBPM]
    @constraint(model, 1/12*sum(x[i]*b[i] for i=1:N) >= DBPM_q75);
    
    # Defensive Win Shares Constraint
    w = df[!,:DWS]
    @constraint(model, 1/12*sum(x[i]*w[i] for i=1:N) >= DWS_q75);

    optimize!(model)
    
    # Extract Selected Players
    players = sort(df[value.(x).>0.99,:Player])
    return(df[value.(x).>0.99,[:Player,:Pos,:MP,:eFGp,:P3p,:P3,:PTS,:DRB,:DWS,:DBPM,:Salary2019_20]])
end

basketize_Raptors (generic function with 2 methods)

### Optimal Raptors Roster 

In [508]:
# Extract Model Selected Players and Their Statistics
basketized_Raptors = basketize_Raptors(playerPool)

,Player,Pos,MP,eFGp,P3p,P3,PTS,DRB,DWS
,String,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Kyle Lowry,PG,36.6,0.52,0.354,2.9,19.7,4.3,2.8
2,Matt Thomas,SG,9.7,0.644,0.467,1.1,4.5,1.2,0.4
3,Norman Powell,SG,28.8,0.593,0.398,2.1,16.4,3.1,1.9
4,OG Anunoby,SF,30.1,0.584,0.381,1.3,10.7,4.2,3.3
5,Pascal Siakam,PF,35.5,0.517,0.359,2.2,23.6,6.5,3.2
6,Patrick McCaw,SF,24.5,0.485,0.324,0.6,4.6,1.8,1.2
7,Stanley Johnson,PF,4.8,0.29,0.154,0.1,1.2,1.0,0.2
8,Terence Davis,SG,17.0,0.575,0.396,1.4,7.7,2.7,1.6
9,Christian Wood,PF,21.4,0.62,0.386,0.9,13.1,4.6,1.5


According to the model, the free agents the Toronto Raptors should target are **Christian Wood, Dwight Howard, Fred Vanvleet** and **Meyers Leonard**. Interestingly, Vanvleet plays for the Raptors so the model is essentially suggesting that the team should resign him. 

It intuitively makes sense that these four players were selected given the parameters of my model since:
* Wood, Vanvleet, and Leonard shoot a decent amount of threes at a high percentage
* They all shoot efficiently (high effective field goal percentage)
* Wood, Howard, and Leonard get a lot of rebounds
* Vanvleet scores a lot of points
* They are all relatively good defenders as they have a decent DWS and DBPM.

In [509]:
# Compute Basketized Raptors Team Payroll
sum(basketized_Raptors.Salary2019_20)

8.24371e7

The payroll of the Raptors with the 4 suggested players would be approximately \\$82 million, saving the team about \\$41 million compared to their 2019-2020 payroll. This will give the team flexibility to do numerous things. For example, the extra \\$41 million can partially be allocated towards increasing the salary of the 4 suggested players from their 2019-2020 salary, especially Vanvleet and Wood who appear to be heavily undervalued.

#   
# Model Limitations

A limitation with my model and most models related to roster creation is it does not take into account how the players will play together and how adjusting their minutes could impact their output (performance statistics). Another limitation of my model is the 3-point percentage (P3p) and effective field-goal percentage (eFGp) constraints assume that every player on a team shoots the same amount which is not true. The model looks for the average P3p and eFGp across the 12 players which is slightly different from a team's P3p and eFGp. Essentially, the shooting percentage of a player who shoots a lot should be weighted more when calculating a team's shooting percentage than a player who rarely shoots the ball.

#    
# Conclusion 

In conclusion, winning teams tend to score a lot of points, play good defense, rebound the ball well, and shoot efficiently. The Basketize model takes all of these factors into account when optimizing a roster. NBA teams can use this model not only to determine who they should target in free agency, but also to determine which players they should trade away/for to improve their team's statistics. In addition, this model can be tailored to a team's needs and style of play. For example, a team looking to improve defensively can set the objective function to maximize Defensive Win Shares (DWS) or Defensive Box Plus Minus (DBPM) and set a payroll constraint to be under the salary cap. Overall, NBA teams can use this model for a wide variety of applications.